In [1]:
%matplotlib inline
# Import libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path

# Make pandas dataframes prettier
from IPython.display import display, HTML

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
from google.cloud import storage
auth.authenticate_user()
client = bigquery.Client(project='physionet-data')
datasets = client.list_datasets()
project_id='isentropic-keep-363112'
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id

## Demographic Feature Extraction

In [2]:
# Age, Sex, Ethnicity, and Length of stay (hospital and ICU)
%%bigquery demo_df
SELECT ie.subject_id, ie.hadm_id, ie.stay_id

-- patient level factors
, pat.gender, pat.dod

-- hospital level factors
, adm.admittime, adm.dischtime
#LOS_HOSPTIAL
, DATETIME_DIFF(adm.dischtime, adm.admittime, DAY) as los_hospital
#AGE
, DATETIME_DIFF(adm.admittime, DATETIME(pat.anchor_year, 1, 1, 0, 0, 0), YEAR) + pat.anchor_age as admission_age
, adm.race
, adm.hospital_expire_flag
, DENSE_RANK() OVER (PARTITION BY adm.subject_id ORDER BY adm.admittime) AS hospstay_seq
, CASE
    WHEN DENSE_RANK() OVER (PARTITION BY adm.subject_id ORDER BY adm.admittime) = 1 THEN True
    ELSE False END AS first_hosp_stay

-- icu level factors
, ie.intime as icu_intime, ie.outtime as icu_outtime
, ROUND(CAST(DATETIME_DIFF(ie.outtime, ie.intime, HOUR)/24.0 AS NUMERIC), 2) as los_icu
, DENSE_RANK() OVER (PARTITION BY ie.hadm_id ORDER BY ie.intime) AS icustay_seq

-- first ICU stay *for the current hospitalization*
, CASE
    WHEN DENSE_RANK() OVER (PARTITION BY ie.hadm_id ORDER BY ie.intime) = 1 THEN True
    ELSE False END AS first_icu_stay

FROM `physionet-data.mimiciv_icu.icustays` ie
INNER JOIN `physionet-data.mimiciv_hosp.admissions` adm
    ON ie.hadm_id = adm.hadm_id
INNER JOIN `physionet-data.mimiciv_hosp.patients` pat
    ON ie.subject_id = pat.subject_id

Query is running:   0%|          |

Downloading:   0%|          |

In [3]:
demo_df

,subject_id,hadm_id,stay_id,gender,dod,admittime,dischtime,los_hospital,admission_age,race,hospital_expire_flag,hospstay_seq,first_hosp_stay,icu_intime,icu_outtime,los_icu,icustay_seq,first_icu_stay
0,19669999,20005479,32977919,F,2148-08-10,2148-06-01 12:48:00,2148-06-07 18:38:00,6,89,OTHER,0,1,True,2148-06-01 14:07:00,2148-06-03 18:30:09,2.170000000,1,True
1,17002995,20006309,31646901,M,NaT,2177-12-02 01:39:00,2177-12-09 19:10:00,7,51,WHITE - BRAZILIAN,0,2,False,2177-12-02 02:36:00,2177-12-06 16:27:04,4.580000000,1,True
2,10559183,20008400,36107959,M,2116-11-14,2116-01-03 14:50:00,2116-01-07 16:32:00,4,91,WHITE,0,1,True,2116-01-03 16:00:00,2116-01-06 18:44:57,3.080000000,1,True
3,18172155,20009330,36841282,M,NaT,2144-01-01 00:33:00,2144-01-09 21:07:00,8,73,ASIAN - SOUTH EAST ASIAN,0,2,False,2144-01-01 04:26:36,2144-01-06 22:46:18,5.750000000,1,True
4,18549459,20021612,34145253,F,NaT,2162-05-06 17:35:00,2162-05-08 14:30:00,2,47,BLACK/AFRICAN AMERICAN,0,3,False,2162-05-06 19:23:50,2162-05-08 00:14:57,1.210000000,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76938,13883852,29836978,33958669,F,2110-12-08,2110-12-05 20:53:00,2110-12-08 14:49:00,3,91,WHITE,1,1,True,2110-12-07 00:35:34,2110-12-08 14:50:30,1.580000000,1,True
76939,12658393,29839743,35094892,F,NaT,2190-04-30 19:24:00,2190-05-03 15:11:00,3,80,WHITE,0,1,True,2190-04-30 20:35:00,2190-05-01 22:03:29,1.080000000,1,True
76940,10573068,29843410,38564101,F,NaT,2153-03-27 00:18:00,2153-04-03 17:28:00,7,46,UNKNOWN,0,1,True,2153-03-27 00:19:00,2153-03-31 23:30:49,4.960000000,1,True
76941,16563332,29988360,39524830,F,2162-01-01,2159-05-28 18:06:00,2159-06-03 17:35:00,6,56,WHITE,0,1,True,2159-05-29 14:57:44,2159-05-30 15:51:31,1.040000000,1,True


## Lab Vitals Extraction


In [4]:
# Create dictionary of ranges
lab_ranges = {'BICARBONATE': [22,32],
              'BUN': [6,20],
              'CALCIUM': [8.4,10.3],
              'CHLORIDE': [96,108],
              'CREATININE': [0.4,1.1],
              'HEMOGLOBIN': [11.2,15.7],
              'LACTATE': [0.5,2.0],
              'MAGNESIUM': [1.6,2.6],
              'PHOSPHATE': [2.7,4.5],
              'PLATELET': [150,400],
              'POTASSIUM': [3.3,5.1],
              'SODIUM': [133,145]
             }

In [5]:
%%bigquery df_lab
WITH pvt AS (
  SELECT ie.subject_id, ie.hadm_id, ie.outtime, ie.stay_id, le.charttime, ad.deathtime
  , CASE when ad.deathtime between ie.intime and ie.outtime and (ie.outtime -  ie.intime < interval '30' day) THEN 1 ELSE 0 END AS mort_icu
  , CASE when ad.deathtime between ad.admittime and ad.dischtime and (ad.dischtime -  ad.admittime < interval '30' day) THEN 1 ELSE 0 END AS mort_hosp
  -- here we assign labels to ITEMIDs
  -- this also fuses together multiple ITEMIDs containing the same data
  , CASE
        when le.itemid = 50868 then 'ANION GAP'
        when le.itemid = 50862 then 'ALBUMIN'
        when le.itemid = 50882 then 'BICARBONATE'
        when le.itemid = 50885 then 'BILIRUBIN'
        when le.itemid = 50912 then 'CREATININE'
        when le.itemid = 50806 then 'CHLORIDE'
        when le.itemid = 50902 then 'CHLORIDE'
        when itemid = 50809 then 'GLUCOSE'
        when itemid = 50931 then 'GLUCOSE'
        when itemid = 50810 then 'HEMATOCRIT'
        when itemid = 51221 then 'HEMATOCRIT'
        when itemid = 50811 then 'HEMOGLOBIN'
        when itemid = 51222 then 'HEMOGLOBIN'
        when itemid = 50813 then 'LACTATE'
        when itemid = 50960 then 'MAGNESIUM'
        when itemid = 50970 then 'PHOSPHATE'
        when itemid = 51265 then 'PLATELET'
        when itemid = 50822 then 'POTASSIUM'
        when itemid = 50971 then 'POTASSIUM'
        when itemid = 51275 then 'PTT'
        when itemid = 51237 then 'INR'
        when itemid = 51274 then 'PT'
        when itemid = 50824 then 'SODIUM'
        when itemid = 50983 then 'SODIUM'
        when itemid = 51006 then 'BUN'
        when itemid = 51300 then 'WBC'
        when itemid = 51301 then 'WBC'
        when itemid = 51108 then 'URINEOUTPUT'
      ELSE null
      END AS label
  , -- add in some sanity checks on the values
    -- the where clause below requires all valuenum to be > 0, 
    -- so these are only upper limit checks
    CASE
      when le.itemid = 50862 and le.valuenum >    10 then null -- g/dL 'ALBUMIN'
      when le.itemid = 50868 and le.valuenum > 10000 then null -- mEq/L 'ANION GAP'
      when le.itemid = 50882 and le.valuenum > 10000 then null -- mEq/L 'BICARBONATE'
      when le.itemid = 50885 and le.valuenum >   150 then null -- mg/dL 'BILIRUBIN'
      when le.itemid = 50806 and le.valuenum > 10000 then null -- mEq/L 'CHLORIDE'
      when le.itemid = 50902 and le.valuenum > 10000 then null -- mEq/L 'CHLORIDE'
      when le.itemid = 50912 and le.valuenum >   150 then null -- mg/dL 'CREATININE'
      when le.itemid = 50809 and le.valuenum > 10000 then null -- mg/dL 'GLUCOSE'
      when le.itemid = 50931 and le.valuenum > 10000 then null -- mg/dL 'GLUCOSE'
      when le.itemid = 50810 and le.valuenum >   100 then null -- % 'HEMATOCRIT'
      when le.itemid = 51221 and le.valuenum >   100 then null -- % 'HEMATOCRIT'
      when le.itemid = 50811 and le.valuenum >    50 then null -- g/dL 'HEMOGLOBIN'
      when le.itemid = 51222 and le.valuenum >    50 then null -- g/dL 'HEMOGLOBIN'
      when le.itemid = 50813 and le.valuenum >    50 then null -- mmol/L 'LACTATE'
      when le.itemid = 50960 and le.valuenum >    60 then null -- mmol/L 'MAGNESIUM'
      when le.itemid = 50970 and le.valuenum >    60 then null -- mg/dL 'PHOSPHATE'
      when le.itemid = 51265 and le.valuenum > 10000 then null -- K/uL 'PLATELET'
      when le.itemid = 50822 and le.valuenum >    30 then null -- mEq/L 'POTASSIUM'
      when le.itemid = 50971 and le.valuenum >    30 then null -- mEq/L 'POTASSIUM'
      when le.itemid = 51275 and le.valuenum >   150 then null -- sec 'PTT'
      when le.itemid = 51237 and le.valuenum >    50 then null -- 'INR'
      when le.itemid = 51274 and le.valuenum >   150 then null -- sec 'PT'
      when le.itemid = 50824 and le.valuenum >   200 then null -- mEq/L == mmol/L 'SODIUM'
      when le.itemid = 50983 and le.valuenum >   200 then null -- mEq/L == mmol/L 'SODIUM'
      when le.itemid = 51006 and le.valuenum >   300 then null -- 'BUN'
      when le.itemid = 51300 and le.valuenum >  1000 then null -- 'WBC'
      when le.itemid = 51301 and le.valuenum >  1000 then null -- 'WBC'
      when le.itemid = 51108 and le.valuenum >  10000 then null -- 'URINEOUTPUT'
    ELSE le.valuenum
    END AS valuenum
  FROM `physionet-data.mimiciv_icu.icustays` ie

  LEFT JOIN `physionet-data.mimiciv_hosp.labevents` le
    ON le.subject_id = ie.subject_id 
    AND le.hadm_id = ie.hadm_id
    AND le.charttime between (ie.intime - interval '6' hour) 
    AND (ie.intime + interval '1' day)
    AND le.itemid IN
    (
      -- comment is: LABEL | CATEGORY | FLUID | NUMBER OF ROWS IN LABEVENTS
      50868, -- ANION GAP | CHEMISTRY | BLOOD | 769895
      50862, -- ALBUMIN | CHEMISTRY | BLOOD | 146697
      50882, -- BICARBONATE | CHEMISTRY | BLOOD | 780733
      50885, -- BILIRUBIN, TOTAL | CHEMISTRY | BLOOD | 238277
      50912, -- CREATININE | CHEMISTRY | BLOOD | 797476
      50902, -- CHLORIDE | CHEMISTRY | BLOOD | 795568
      50806, -- CHLORIDE, WHOLE BLOOD | BLOOD GAS | BLOOD | 48187
      50931, -- GLUCOSE | CHEMISTRY | BLOOD | 748981
      50809, -- GLUCOSE | BLOOD GAS | BLOOD | 196734
      51221, -- HEMATOCRIT | HEMATOLOGY | BLOOD | 881846
      50810, -- HEMATOCRIT, CALCULATED | BLOOD GAS | BLOOD | 89715
      51222, -- HEMOGLOBIN | HEMATOLOGY | BLOOD | 752523
      50811, -- HEMOGLOBIN | BLOOD GAS | BLOOD | 89712
      50813, -- LACTATE | BLOOD GAS | BLOOD | 187124
      50960, -- MAGNESIUM | CHEMISTRY | BLOOD | 664191
      50970, -- PHOSPHATE | CHEMISTRY | BLOOD | 590524
      51265, -- PLATELET COUNT | HEMATOLOGY | BLOOD | 778444
      50971, -- POTASSIUM | CHEMISTRY | BLOOD | 845825
      50822, -- POTASSIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 192946
      51275, -- PTT | HEMATOLOGY | BLOOD | 474937
      51237, -- INR(PT) | HEMATOLOGY | BLOOD | 471183
      51274, -- PT | HEMATOLOGY | BLOOD | 469090
      50983, -- SODIUM | CHEMISTRY | BLOOD | 808489
      50824, -- SODIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 71503
      51006, -- UREA NITROGEN | CHEMISTRY | BLOOD | 791925
      51301, -- WHITE BLOOD CELLS | HEMATOLOGY | BLOOD | 753301
      51300,  -- WBC COUNT | HEMATOLOGY | BLOOD | 2371
      51108  -- URINE VOLUME | CHEMISTRY | URINE | 11061
    )
    AND le.valuenum IS NOT null 
    AND le.valuenum > 0 -- lab values cannot be 0 and cannot be negative
    
    LEFT JOIN `physionet-data.mimiciv_hosp.admissions` ad
    ON ie.subject_id = ad.subject_id
    AND ie.hadm_id = ad.hadm_id
    
    -- WHERE ie.subject_id < 10000
    
) 

SELECT r.subject_id, r.hadm_id, r.stay_id, r.mort_icu, r.mort_hosp
  , min(case when label = 'ANION GAP' then valuenum else null end) as aniongap_min
  , max(case when label = 'ANION GAP' then valuenum else null end) as aniongap_max
  , min(case when label = 'ALBUMIN' then valuenum else null end) as albumin_min
  , max(case when label = 'ALBUMIN' then valuenum else null end) as albumin_max
  , min(case when label = 'BICARBONATE' then valuenum else null end) as bicarbonate_min
  , max(case when label = 'BICARBONATE' then valuenum else null end) as bicarbonate_max
  , min(case when label = 'BILIRUBIN' then valuenum else null end) as bilirubin_min
  , max(case when label = 'BILIRUBIN' then valuenum else null end) as bilirubin_max
  , min(case when label = 'CREATININE'then valuenum else null end) as creatinine_min
  , max(case when label = 'CREATININE'then valuenum else null end) as creatinine_max
  , min(case when label = 'CHLORIDE' then valuenum else null end) as chloride_min
  , max(case when label = 'CHLORIDE' then valuenum else null end) as chloride_max
  , min(case when label = 'HEMATOCRIT' then valuenum else null end) as hematocrit_min
  , max(case when label = 'HEMATOCRIT' then valuenum else null end) as hematocrit_max
  , min(case when label = 'HEMOGLOBIN' then valuenum else null end) as hemoglobin_min
  , max(case when label = 'HEMOGLOBIN' then valuenum else null end) as hemoglobin_max
  , min(case when label = 'LACTATE' then valuenum else null end) as lactate_min
  , max(case when label = 'LACTATE' then valuenum else null end) as lactate_max
  , Avg(case when label = 'LACTATE' then valuenum else null end) as lactate_mean
  , min(case when label = 'MAGNESIUM' then valuenum else null end) as magnesium_min
  , max(case when label = 'MAGNESIUM' then valuenum else null end) as magnesium_max
  , min(case when label = 'PHOSPHATE' then valuenum else null end) as phosphate_min
  , max(case when label = 'PHOSPHATE' then valuenum else null end) as phosphate_max
  , min(case when label = 'PLATELET' then valuenum else null end) as platelet_min
  , max(case when label = 'PLATELET' then valuenum else null end) as platelet_max
  , min(case when label = 'POTASSIUM' then valuenum else null end) as potassium_min
  , max(case when label = 'POTASSIUM' then valuenum else null end) as potassium_max
  , min(case when label = 'PTT' then valuenum else null end) as ptt_min
  , max(case when label = 'PTT' then valuenum else null end) as ptt_max
  , min(case when label = 'INR' then valuenum else null end) as inr_min
  , max(case when label = 'INR' then valuenum else null end) as inr_max
  , min(case when label = 'PT' then valuenum else null end) as pt_min
  , max(case when label = 'PT' then valuenum else null end) as pt_max
  , min(case when label = 'SODIUM' then valuenum else null end) as sodium_min
  , max(case when label = 'SODIUM' then valuenum else null end) as sodium_max
  , min(case when label = 'BUN' then valuenum else null end) as bun_min
  , max(case when label = 'BUN' then valuenum else null end) as bun_max
  , AVG(case when label = 'BUN' then valuenum else null end) as bun_mean
  , min(case when label = 'WBC' then valuenum else null end) as wbc_min
  , max(case when label = 'WBC' then valuenum else null end) as wbc_max
  , Avg(case when label = 'WBC' then valuenum else null end) as wbc_mean
  , max(case when label = 'URINEOUTPUT' then valuenum else null end) as urineoutput

FROM pvt r
GROUP BY r.subject_id, r.hadm_id, r.stay_id, r.mort_icu, r.mort_hosp 
ORDER BY r.subject_id, r.hadm_id, r.stay_id, r.mort_icu, r.mort_hosp



Query is running:   0%|          |

Downloading:   0%|          |

In [6]:
df_labs = pd.merge(demo_df, df_lab, how = 'left', on = ['stay_id', 'subject_id', 'hadm_id'])

In [7]:
df_labs

,subject_id,hadm_id,stay_id,gender,dod,admittime,dischtime,los_hospital,admission_age,race,...,pt_max,sodium_min,sodium_max,bun_min,bun_max,bun_mean,wbc_min,wbc_max,wbc_mean,urineoutput
0,19669999,20005479,32977919,F,2148-08-10,2148-06-01 12:48:00,2148-06-07 18:38:00,6,89,OTHER,...,16.8,136.0,136.0,21.0,22.0,21.500000,20.9,27.9,24.400000,NaN
1,17002995,20006309,31646901,M,NaT,2177-12-02 01:39:00,2177-12-09 19:10:00,7,51,WHITE - BRAZILIAN,...,11.4,137.0,137.0,15.0,24.0,19.500000,9.4,15.0,12.200000,NaN
2,10559183,20008400,36107959,M,2116-11-14,2116-01-03 14:50:00,2116-01-07 16:32:00,4,91,WHITE,...,13.8,133.0,136.0,20.0,22.0,21.000000,13.5,13.5,13.500000,NaN
3,18172155,20009330,36841282,M,NaT,2144-01-01 00:33:00,2144-01-09 21:07:00,8,73,ASIAN - SOUTH EAST ASIAN,...,15.4,139.0,140.0,17.0,19.0,17.666667,7.8,8.9,8.466667,NaN
4,18549459,20021612,34145253,F,NaT,2162-05-06 17:35:00,2162-05-08 14:30:00,2,47,BLACK/AFRICAN AMERICAN,...,NaN,138.0,138.0,62.0,62.0,62.000000,6.4,6.4,6.400000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76938,13883852,29836978,33958669,F,2110-12-08,2110-12-05 20:53:00,2110-12-08 14:49:00,3,91,WHITE,...,12.4,117.0,122.0,45.0,52.0,49.250000,18.5,18.5,18.500000,NaN
76939,12658393,29839743,35094892,F,NaT,2190-04-30 19:24:00,2190-05-03 15:11:00,3,80,WHITE,...,14.1,140.0,140.0,8.0,8.0,8.000000,4.7,4.7,4.700000,NaN
76940,10573068,29843410,38564101,F,NaT,2153-03-27 00:18:00,2153-04-03 17:28:00,7,46,UNKNOWN,...,13.6,135.0,139.0,3.0,3.0,3.000000,13.9,15.3,14.600000,NaN
76941,16563332,29988360,39524830,F,2162-01-01,2159-05-28 18:06:00,2159-06-03 17:35:00,6,56,WHITE,...,NaN,135.0,142.0,7.0,8.0,7.500000,9.2,12.9,11.050000,NaN


In [18]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [19]:
df_labs.to_csv("/content/drive/My Drive/260 Final Project/df_labs.csv")

## Select only subjects first stay in the ICU for over 24 hours


In [8]:
ICU_df = df_labs[df_labs['first_icu_stay']==True]

In [9]:
ICU_df = ICU_df[ICU_df['los_icu'] > 1]

In [10]:
ICU_df

,subject_id,hadm_id,stay_id,gender,dod,admittime,dischtime,los_hospital,admission_age,race,...,pt_max,sodium_min,sodium_max,bun_min,bun_max,bun_mean,wbc_min,wbc_max,wbc_mean,urineoutput
0,19669999,20005479,32977919,F,2148-08-10,2148-06-01 12:48:00,2148-06-07 18:38:00,6,89,OTHER,...,16.8,136.0,136.0,21.0,22.0,21.500000,20.9,27.9,24.400000,NaN
1,17002995,20006309,31646901,M,NaT,2177-12-02 01:39:00,2177-12-09 19:10:00,7,51,WHITE - BRAZILIAN,...,11.4,137.0,137.0,15.0,24.0,19.500000,9.4,15.0,12.200000,NaN
2,10559183,20008400,36107959,M,2116-11-14,2116-01-03 14:50:00,2116-01-07 16:32:00,4,91,WHITE,...,13.8,133.0,136.0,20.0,22.0,21.000000,13.5,13.5,13.500000,NaN
3,18172155,20009330,36841282,M,NaT,2144-01-01 00:33:00,2144-01-09 21:07:00,8,73,ASIAN - SOUTH EAST ASIAN,...,15.4,139.0,140.0,17.0,19.0,17.666667,7.8,8.9,8.466667,NaN
4,18549459,20021612,34145253,F,NaT,2162-05-06 17:35:00,2162-05-08 14:30:00,2,47,BLACK/AFRICAN AMERICAN,...,NaN,138.0,138.0,62.0,62.0,62.000000,6.4,6.4,6.400000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76938,13883852,29836978,33958669,F,2110-12-08,2110-12-05 20:53:00,2110-12-08 14:49:00,3,91,WHITE,...,12.4,117.0,122.0,45.0,52.0,49.250000,18.5,18.5,18.500000,NaN
76939,12658393,29839743,35094892,F,NaT,2190-04-30 19:24:00,2190-05-03 15:11:00,3,80,WHITE,...,14.1,140.0,140.0,8.0,8.0,8.000000,4.7,4.7,4.700000,NaN
76940,10573068,29843410,38564101,F,NaT,2153-03-27 00:18:00,2153-04-03 17:28:00,7,46,UNKNOWN,...,13.6,135.0,139.0,3.0,3.0,3.000000,13.9,15.3,14.600000,NaN
76941,16563332,29988360,39524830,F,2162-01-01,2159-05-28 18:06:00,2159-06-03 17:35:00,6,56,WHITE,...,NaN,135.0,142.0,7.0,8.0,7.500000,9.2,12.9,11.050000,NaN


## Filter subjects with no demographic information

In [11]:
ICU_df = ICU_df.dropna(subset=['admission_age', 'race', 'gender'], how='all')

In [12]:
ICU_df

,subject_id,hadm_id,stay_id,gender,dod,admittime,dischtime,los_hospital,admission_age,race,...,pt_max,sodium_min,sodium_max,bun_min,bun_max,bun_mean,wbc_min,wbc_max,wbc_mean,urineoutput
0,19669999,20005479,32977919,F,2148-08-10,2148-06-01 12:48:00,2148-06-07 18:38:00,6,89,OTHER,...,16.8,136.0,136.0,21.0,22.0,21.500000,20.9,27.9,24.400000,NaN
1,17002995,20006309,31646901,M,NaT,2177-12-02 01:39:00,2177-12-09 19:10:00,7,51,WHITE - BRAZILIAN,...,11.4,137.0,137.0,15.0,24.0,19.500000,9.4,15.0,12.200000,NaN
2,10559183,20008400,36107959,M,2116-11-14,2116-01-03 14:50:00,2116-01-07 16:32:00,4,91,WHITE,...,13.8,133.0,136.0,20.0,22.0,21.000000,13.5,13.5,13.500000,NaN
3,18172155,20009330,36841282,M,NaT,2144-01-01 00:33:00,2144-01-09 21:07:00,8,73,ASIAN - SOUTH EAST ASIAN,...,15.4,139.0,140.0,17.0,19.0,17.666667,7.8,8.9,8.466667,NaN
4,18549459,20021612,34145253,F,NaT,2162-05-06 17:35:00,2162-05-08 14:30:00,2,47,BLACK/AFRICAN AMERICAN,...,NaN,138.0,138.0,62.0,62.0,62.000000,6.4,6.4,6.400000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76938,13883852,29836978,33958669,F,2110-12-08,2110-12-05 20:53:00,2110-12-08 14:49:00,3,91,WHITE,...,12.4,117.0,122.0,45.0,52.0,49.250000,18.5,18.5,18.500000,NaN
76939,12658393,29839743,35094892,F,NaT,2190-04-30 19:24:00,2190-05-03 15:11:00,3,80,WHITE,...,14.1,140.0,140.0,8.0,8.0,8.000000,4.7,4.7,4.700000,NaN
76940,10573068,29843410,38564101,F,NaT,2153-03-27 00:18:00,2153-04-03 17:28:00,7,46,UNKNOWN,...,13.6,135.0,139.0,3.0,3.0,3.000000,13.9,15.3,14.600000,NaN
76941,16563332,29988360,39524830,F,2162-01-01,2159-05-28 18:06:00,2159-06-03 17:35:00,6,56,WHITE,...,NaN,135.0,142.0,7.0,8.0,7.500000,9.2,12.9,11.050000,NaN


## Filter subjects with no laboratory data

In [13]:
# remove patients with no lab results
ICU_df = ICU_df.dropna(subset=['aniongap_min',
                         'aniongap_max',
                         'albumin_min',
                         'albumin_max',
                         'bicarbonate_min',
                         'bicarbonate_max',
                         'bilirubin_min',
                         'bilirubin_max',
                         'creatinine_min',
                         'creatinine_max',
                         'chloride_min',
                         'chloride_max',
                         'hematocrit_min',
                         'hematocrit_max',
                         'hemoglobin_min',
                         'hemoglobin_max',
                         'lactate_min',
                         'lactate_max',
                         'lactate_mean',
                         'magnesium_min',
                         'magnesium_max',
                         'phosphate_min',
                         'phosphate_max',
                         'platelet_min',
                         'platelet_max',
                         'potassium_min',
                         'potassium_max',
                         'ptt_min',
                         'ptt_max',
                         'inr_min',
                         'inr_max',
                         'pt_min',
                         'pt_max',
                         'sodium_min',
                         'sodium_max',
                         'bun_min',
                         'bun_max',
                         'bun_mean',
                         'wbc_min',
                         'wbc_max',
                         'wbc_mean'], how = 'all')


In [14]:
ICU_df

,subject_id,hadm_id,stay_id,gender,dod,admittime,dischtime,los_hospital,admission_age,race,...,pt_max,sodium_min,sodium_max,bun_min,bun_max,bun_mean,wbc_min,wbc_max,wbc_mean,urineoutput
0,19669999,20005479,32977919,F,2148-08-10,2148-06-01 12:48:00,2148-06-07 18:38:00,6,89,OTHER,...,16.8,136.0,136.0,21.0,22.0,21.500000,20.9,27.9,24.400000,NaN
1,17002995,20006309,31646901,M,NaT,2177-12-02 01:39:00,2177-12-09 19:10:00,7,51,WHITE - BRAZILIAN,...,11.4,137.0,137.0,15.0,24.0,19.500000,9.4,15.0,12.200000,NaN
2,10559183,20008400,36107959,M,2116-11-14,2116-01-03 14:50:00,2116-01-07 16:32:00,4,91,WHITE,...,13.8,133.0,136.0,20.0,22.0,21.000000,13.5,13.5,13.500000,NaN
3,18172155,20009330,36841282,M,NaT,2144-01-01 00:33:00,2144-01-09 21:07:00,8,73,ASIAN - SOUTH EAST ASIAN,...,15.4,139.0,140.0,17.0,19.0,17.666667,7.8,8.9,8.466667,NaN
4,18549459,20021612,34145253,F,NaT,2162-05-06 17:35:00,2162-05-08 14:30:00,2,47,BLACK/AFRICAN AMERICAN,...,NaN,138.0,138.0,62.0,62.0,62.000000,6.4,6.4,6.400000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76938,13883852,29836978,33958669,F,2110-12-08,2110-12-05 20:53:00,2110-12-08 14:49:00,3,91,WHITE,...,12.4,117.0,122.0,45.0,52.0,49.250000,18.5,18.5,18.500000,NaN
76939,12658393,29839743,35094892,F,NaT,2190-04-30 19:24:00,2190-05-03 15:11:00,3,80,WHITE,...,14.1,140.0,140.0,8.0,8.0,8.000000,4.7,4.7,4.700000,NaN
76940,10573068,29843410,38564101,F,NaT,2153-03-27 00:18:00,2153-04-03 17:28:00,7,46,UNKNOWN,...,13.6,135.0,139.0,3.0,3.0,3.000000,13.9,15.3,14.600000,NaN
76941,16563332,29988360,39524830,F,2162-01-01,2159-05-28 18:06:00,2159-06-03 17:35:00,6,56,WHITE,...,NaN,135.0,142.0,7.0,8.0,7.500000,9.2,12.9,11.050000,NaN


## Filter by 20% missing data

In [15]:
ICU_df['full_count'] = ICU_df.apply(lambda x: x.count(), axis=1)

<ipython-input-15-650a4962c50d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ICU_df['full_count'] = ICU_df.apply(lambda x: x.count(), axis=1)


In [16]:
ICU_df = ICU_df[ICU_df['full_count']>48]

In [17]:
ICU_df

,subject_id,hadm_id,stay_id,gender,dod,admittime,dischtime,los_hospital,admission_age,race,...,sodium_min,sodium_max,bun_min,bun_max,bun_mean,wbc_min,wbc_max,wbc_mean,urineoutput,full_count
0,19669999,20005479,32977919,F,2148-08-10,2148-06-01 12:48:00,2148-06-07 18:38:00,6,89,OTHER,...,136.0,136.0,21.0,22.0,21.500000,20.9,27.9,24.400000,NaN,59
1,17002995,20006309,31646901,M,NaT,2177-12-02 01:39:00,2177-12-09 19:10:00,7,51,WHITE - BRAZILIAN,...,137.0,137.0,15.0,24.0,19.500000,9.4,15.0,12.200000,NaN,53
2,10559183,20008400,36107959,M,2116-11-14,2116-01-03 14:50:00,2116-01-07 16:32:00,4,91,WHITE,...,133.0,136.0,20.0,22.0,21.000000,13.5,13.5,13.500000,NaN,58
3,18172155,20009330,36841282,M,NaT,2144-01-01 00:33:00,2144-01-09 21:07:00,8,73,ASIAN - SOUTH EAST ASIAN,...,139.0,140.0,17.0,19.0,17.666667,7.8,8.9,8.466667,NaN,60
4,18549459,20021612,34145253,F,NaT,2162-05-06 17:35:00,2162-05-08 14:30:00,2,47,BLACK/AFRICAN AMERICAN,...,138.0,138.0,62.0,62.0,62.000000,6.4,6.4,6.400000,NaN,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76938,13883852,29836978,33958669,F,2110-12-08,2110-12-05 20:53:00,2110-12-08 14:49:00,3,91,WHITE,...,117.0,122.0,45.0,52.0,49.250000,18.5,18.5,18.500000,NaN,59
76939,12658393,29839743,35094892,F,NaT,2190-04-30 19:24:00,2190-05-03 15:11:00,3,80,WHITE,...,140.0,140.0,8.0,8.0,8.000000,4.7,4.7,4.700000,NaN,57
76940,10573068,29843410,38564101,F,NaT,2153-03-27 00:18:00,2153-04-03 17:28:00,7,46,UNKNOWN,...,135.0,139.0,3.0,3.0,3.000000,13.9,15.3,14.600000,NaN,60
76941,16563332,29988360,39524830,F,2162-01-01,2159-05-28 18:06:00,2159-06-03 17:35:00,6,56,WHITE,...,135.0,142.0,7.0,8.0,7.500000,9.2,12.9,11.050000,NaN,51


## Export Data to Google Drive to be downloaded

In [99]:
#Acces to read and write files from project google drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [101]:
ICU_df.to_csv("/content/drive/My Drive/260 Final Project/ICU_df.csv")